In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import geopandas as gpd  # For handling GeoJSON data
import folium
import branca.colormap as cm
from streamlit_folium import folium_static  # Import to render folium maps in Streamlit
from sklearn.metrics import accuracy_score

2024-10-16 11:37:31.091 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [2]:
# Load the GeoJSON file
zip_geojson = gpd.read_file('demo_zips.geojson')

# Load person data, forcing ZIP to be read as strings
person_data = pd.read_csv('data.csv', dtype={'ZIP': str})

# Ensure ZIP codes have leading zeros and handle floats
person_data['ZIP'] = person_data['ZIP'].apply(lambda x: str(int(float(x))).zfill(5) if pd.notnull(x) else '')

# Ensure GeoJSON ZIP codes are formatted as strings with leading zeros
zip_geojson['ZIP'] = zip_geojson['ZCTA5CE10'].astype(str).str.zfill(5)


In [20]:
energy_score_threshold=0.5

In [34]:
person_data.shape

(196045, 9)

In [41]:
person_data.head()

,PROFILE_ID,NUMBER_OF_TRADELINES,WEIGHTED_ENERGYSCORE,WEIGHTED_ACTUAL_OUTPUT,FICO_V9_SCORE,ZIP,EQUIFAX_TYPE_CODE,FICO_PASS,ENERGYSCORE_PASS
0,1,15,0.061694,0,696,04265,FF,False,False
1,2,13,0.055457,0,557,33467,FP,False,False
2,3,11,0.082060,0,621,02131,DC,False,False
3,4,4,0.000078,0,638,17968,FA,False,False
4,5,6,0.000887,0,563,94118,DM,False,False


In [51]:
person_data[person_data['ZIP']=='01001'].head(15)

,PROFILE_ID,NUMBER_OF_TRADELINES,WEIGHTED_ENERGYSCORE,WEIGHTED_ACTUAL_OUTPUT,FICO_V9_SCORE,ZIP,EQUIFAX_TYPE_CODE,FICO_PASS,ENERGYSCORE_PASS
19926,20140,7,0.000196,0,781,01001,FZ,True,True
21069,21296,4,0.000472,0,699,01001,FA,False,True
53204,53827,6,0.081874,0,758,01001,FZ,True,True
58658,59376,11,0.000896,0,843,01001,BB,True,True
79839,80901,4,0.168476,0,501,01001,FY,False,True
91399,92628,5,0.000360,0,549,01001,VF,False,True
104589,106019,3,0.000132,0,700,01001,BB,False,True
108599,110086,1,0.000038,0,721,01001,BB,True,True
161372,164003,8,0.001151,0,706,01001,FZ,True,True
161413,164044,4,0.000329,0,723,01001,FZ,True,True


In [52]:
7/11

0.6363636363636364

In [35]:
zip_metrics.head()

,ZIP,Total Population,Percent Below FICO,Percent Above FICO,FICO Accuracy,EnergyScore Accuracy,Qualification Increase
0,01001,11.0,0.363636,0.636364,1.0,1.0,36.363636
1,01002,12.0,0.166667,0.833333,1.0,1.0,16.666667
2,01005,3.0,0.000000,1.000000,NaN,NaN,0.000000
3,01007,18.0,0.222222,0.777778,1.0,1.0,22.222222
4,01010,3.0,0.333333,0.666667,1.0,1.0,33.333333


In [53]:
# Function to calculate metrics for each ZIP
def calculate_zip_metrics(stats_data_person, fico_threshold, energy_score_threshold):
    stats_data_person['FICO_PASS'] = stats_data_person['FICO_V9_SCORE'] > fico_threshold
    stats_data_person['ENERGYSCORE_PASS'] = stats_data_person['WEIGHTED_ENERGYSCORE'] > energy_score_threshold

    def calc_metrics(group):
        total_population = len(group)

        if total_population == 0:
            return pd.Series({
                'Total Population': 0,
                'Percent Below FICO': 0,
                'Percent Above FICO': 0,
                'FICO Accuracy': np.nan,
                'EnergyScore Accuracy': np.nan,
                'Qualification Increase': 0,
            })
        
        above_fico = group[group['FICO_V9_SCORE'] > fico_threshold]
        below_fico = group[group['FICO_V9_SCORE'] < fico_threshold]

        below_fico_pass = below_fico[below_fico['WEIGHTED_ENERGYSCORE'] <= energy_score_threshold]
        
        pct_below_fico = len(below_fico) / total_population
        pct_above_fico = len(above_fico) / total_population

        percent_increase_in_qualifications = (len(below_fico_pass) / total_population) * 100 if len(below_fico_pass) > 0 else 0
        numeric_increase_in_qualifications = len(below_fico_pass) if len(below_fico_pass) > 0 else 0

        energy_accuracy = accuracy_score(group['WEIGHTED_ACTUAL_OUTPUT'], group['ENERGYSCORE_PASS']) #if len(below_fico) > 0 else np.nan

        fico_accuracy = accuracy_score(group['WEIGHTED_ACTUAL_OUTPUT'], group['FICO_PASS']) #if len(below_fico) > 0 else np.nan

        return pd.Series({
            'Total Population': total_population,
            'Percent Below FICO': pct_below_fico,
            'Percent Above FICO': pct_above_fico,
            'FICO Accuracy': fico_accuracy,
            'EnergyScore Accuracy': energy_accuracy,
            'Qualification Increase': percent_increase_in_qualifications,
            'Numeric Increase': numeric_increase_in_qualifications,
        })

    # Group by ZIP and apply metrics calculation
    zip_metrics = stats_data_person.groupby('ZIP').apply(calc_metrics)
    zip_metrics = zip_metrics.reset_index()
    return zip_metrics


# Function to calculate ZIP to utility mapping and display on the map
def calculate_zip_to_util(zip_level_geo, state_name):
    # Load the utility data for the state
    state_util = load_state_util(state_name)
    state_util.rename(columns={'new_name': 'Utility'}, inplace=True)

    # Ensure ZIP code geometries have the same projection as the utility data
    zip_level_geo = zip_level_geo.to_crs(state_util.crs)

    # Convert the ZIP geometries to representative points
    zip_level_geo['geometry'] = zip_level_geo.representative_point()

    # Perform spatial join with utility data based on point locations
    zip_level_geo = gpd.sjoin(zip_level_geo, state_util, how='left', predicate='within')

    # Group by utility name ('new_name') and calculate the mean of 'Qualification Increase'
    zip_to_util = zip_level_geo.groupby('Utility')['Qualification Increase'].mean().reset_index()

    # Merge utility data with the calculated qualification increase
    state_util = state_util.merge(zip_to_util, on='Utility', how='left')

    return state_util


In [47]:
state_name = "Massachusetts"
fico_threshold = 700
energy_score_threshold = .5


In [ ]:
zip_metrics = calculate_zip_metrics(person_data, fico_threshold, energy_score_threshold)
zip_level_geo = pd.merge(zip_metrics, zip_geojson, on='ZIP', how='left')
zip_level_geo = zip_level_geo.dropna(subset=['geometry'])
zip_level_geo = gpd.GeoDataFrame(zip_level_geo, geometry='geometry')


In [49]:
zip_metrics.head()

,ZIP,Total Population,Percent Below FICO,Percent Above FICO,FICO Accuracy,EnergyScore Accuracy,Qualification Increase,Numeric Increase
0,01001,11.0,0.272727,0.636364,1.0,0.0,27.272727,3.0
1,01002,12.0,0.166667,0.833333,1.0,0.0,16.666667,2.0
2,01005,3.0,0.000000,1.000000,NaN,NaN,0.000000,0.0
3,01007,18.0,0.222222,0.777778,1.0,0.0,22.222222,4.0
4,01010,3.0,0.333333,0.666667,1.0,0.0,33.333333,1.0


In [50]:
zip_metrics['Numeric Increase'].describe()

count    23050.000000
mean         3.378655
std          4.289429
min          0.000000
25%          1.000000
50%          2.000000
75%          5.000000
max         43.000000
Name: Numeric Increase, dtype: float64

In [7]:
zip_level_geo.head()

,ZIP,Total Population,Percent Below FICO,Percent Above FICO,FICO Accuracy,EnergyScore Accuracy,Qualification Increase,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,state,geometry
0,01001,11.0,0.181818,0.818182,1.0,1.0,18.181818,01001,01001,B5,G6350,S,29797658.0,2121390.0,+42.0623678,-072.6257536,Massachusetts,"MULTIPOLYGON (((-72.66788 42.04416, -72.66788 ..."
1,01002,12.0,0.166667,0.833333,1.0,1.0,16.666667,01002,01002,B5,G6350,S,142615075.0,4281257.0,+42.3640311,-072.4587594,Massachusetts,"MULTIPOLYGON (((-72.54666 42.40131, -72.54656 ..."
2,01005,3.0,0.000000,1.000000,NaN,NaN,0.000000,01005,01005,B5,G6350,S,114638390.0,666424.0,+42.4188835,-072.1120769,Massachusetts,"MULTIPOLYGON (((-72.19734 42.42264, -72.19661 ..."
3,01007,18.0,0.055556,0.944444,1.0,1.0,5.555556,01007,01007,B5,G6350,S,136237871.0,6941673.0,+42.2790098,-072.4004682,Massachusetts,"MULTIPOLYGON (((-72.47202 42.35151, -72.47138 ..."
4,01010,3.0,0.333333,0.666667,1.0,1.0,33.333333,01010,01010,B5,G6350,S,90055966.0,1421379.0,+42.1281757,-072.2053516,Massachusetts,"MULTIPOLYGON (((-72.27435 42.14364, -72.27411 ..."


In [8]:
zip_level_geo.shape

(644, 18)

In [10]:
zip_level_geo['Total Population'].describe()

count    644.000000
mean       8.427019
std        8.490922
min        1.000000
25%        2.000000
50%        6.000000
75%       12.000000
max       48.000000
Name: Total Population, dtype: float64

In [7]:
import fiona

In [48]:
# Load the GeoJSON file
#zip_geojson = gpd.read_file('select_zips.geojson')

zip_geojson = gpd.read_file('demo_zips.geojson')

# Load person data, forcing ZIP to be read as strings
person_data = pd.read_csv('data.csv', dtype={'ZIP': str})

# Ensure ZIP codes have leading zeros and handle floats
person_data['ZIP'] = person_data['ZIP'].apply(lambda x: str(int(float(x))).zfill(5) if pd.notnull(x) else '')

# Ensure GeoJSON ZIP codes are formatted as strings with leading zeros
zip_geojson['ZIP'] = zip_geojson['ZCTA5CE10'].astype(str).str.zfill(5)

# Check if ZIPs were properly converted
print(person_data['ZIP'].head())
print(zip_geojson['ZIP'].head())


0    04265
1    33467
2    02131
3    17968
4    94118
Name: ZIP, dtype: object
0    02457
1    02458
2    02459
3    02460
4    02461
Name: ZIP, dtype: object


In [49]:
# Function to calculate metrics for each ZIP
def calculate_zip_metrics(stats_data_person, fico_cutoff, energyscore_cutoff):
    # Create masks for conditions
    stats_data_person['FICO_PASS'] = stats_data_person['FICO_V9_SCORE'] > fico_cutoff
    stats_data_person['ENERGYSCORE_PASS'] = stats_data_person['WEIGHTED_ENERGYSCORE'] > energyscore_cutoff

    # Function to calculate various metrics
    def calc_metrics(group):
        total_population = len(group)
        if total_population == 0:
            return pd.Series({
                'Total Population': 0,
                'Percent Below FICO': 0,
                'Percent Above FICO': 0,
                'FICO Accuracy': np.nan,
                'EnergyScore Accuracy': np.nan,
                'Qualification Increase': 0,
            })
        
        
        below_fico = group[group['FICO_PASS'] == False]
        above_fico = group[group['FICO_PASS'] == True]

        if len(below_fico) == 0:
            return pd.Series({
                'Total Population': total_population,
                'Percent Below FICO': 0,
                'Percent Above FICO': len(above_fico) / total_population,
                'FICO Accuracy': np.nan,
                'EnergyScore Accuracy': np.nan,
                'Qualification Increase': 0,
            })

        below_fico_pass = below_fico[below_fico['WEIGHTED_ENERGYSCORE'] <= energyscore_cutoff]
        below_fico_fail = below_fico[below_fico['WEIGHTED_ENERGYSCORE'] > energyscore_cutoff]

        pct_below_fico = len(below_fico) / total_population
        pct_above_fico = len(above_fico) / total_population

        if len(below_fico_pass) == 0:
            percent_increase_in_qualifications = 0
        else:
            percent_increase_in_qualifications = (len(below_fico_pass) / total_population) * 100

        # get FICO accuracy, precision, recall, f1 and roc_auc score
       # fico_accuracy = accuracy_score(below_fico['WEIGHTED_ACTUAL_OUTPUT'], below_fico['WEIGHTED_ENERGYSCORE'] > energyscore_cutoff) if len(below_fico) > 0 else np.nan
        fico_accuracy = accuracy_score(below_fico['WEIGHTED_ACTUAL_OUTPUT'], below_fico['FICO_PASS']) if len(below_fico) > 0 else np.nan

        energy_accuracy = accuracy_score(below_fico['WEIGHTED_ACTUAL_OUTPUT'], below_fico['ENERGYSCORE_PASS']) if len(below_fico) > 0 else np.nan

      #  accuracy_increase = energy_accuracy - fico_accuracy

        return pd.Series({
            'Total Population': total_population,
            'Percent Below FICO': pct_below_fico,
            'Percent Above FICO': pct_above_fico,
            'FICO Accuracy': fico_accuracy,
            'EnergyScore Accuracy': energy_accuracy,
            'Qualification Increase': percent_increase_in_qualifications,
          #  'Accuracy Percentage Increase': accuracy_increase,

           # 'Accuracy Percentage Increase': (energy_accuracy - fico_accuracy) / fico_accuracy * 100 if fico_accuracy * energy_accuracy> 0 else 0,
           

        })

    # Group by ZIP and apply metrics calculation
    zip_metrics = stats_data_person.groupby('ZIP').apply(calc_metrics)

    zip_metrics = zip_metrics.reset_index()
    return zip_metrics

fico_threshold = 700
energy_score_threshold = 0.5
# Calculate metrics and merge with geo data
zip_metrics = calculate_zip_metrics(person_data, fico_threshold, energy_score_threshold)
zip_level_geo = pd.merge(zip_metrics, zip_geojson, on='ZIP', how='left')
zip_level_geo = zip_level_geo.dropna(subset=['geometry'])
zip_level_geo = gpd.GeoDataFrame(zip_level_geo, geometry='geometry')


/var/folders/4_/36zlcy3x3m12nwb2nfhmkkjc0000gp/T/ipykernel_37470/1243698195.py:68: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  zip_metrics = stats_data_person.groupby('ZIP').apply(calc_metrics)


In [ ]:
zip_level_geo

In [ ]:


def load_state_util(state_name):
    if state_name == 'New Mexico':
        temp = gpd.read_file('nm_utils.geojson')
        temp = temp[['new_name', 'geometry']]
        return temp
    elif state_name == 'Massachusetts':
        return gpd.read_file('ma_utils.geojson')


def calculate_zip_to_util(zip_level_geo, util_name, state_name):
    # Load the utility data for the state
    state_util = load_state_util(state_name)

    # Ensure ZIP code geometries have the same projection as the utility data
    zip_level_geo = zip_level_geo.to_crs(state_util.crs)

    # Convert the ZIP geometries to representative points (instead of centroids)
    zip_level_geo['geometry'] = zip_level_geo.representative_point()

    # Perform spatial join with utility data based on point locations using predicate
    zip_level_geo = gpd.sjoin(zip_level_geo, state_util, how='left', predicate='within')

    # Group by utility name ('new_name') and calculate the mean of 'Qualification Increase'
    zip_to_util = zip_level_geo.groupby('new_name')['Qualification Increase'].mean().reset_index()

    return zip_to_util

# Example usage
state_name = 'New Mexico'
util_name = 'Xcel'
zip_to_util = calculate_zip_to_util(zip_level_geo, util_name, state_name)
zip_to_util.head()


In [ ]:
zip_metrics

In [ ]:
zip_level_geo.shape

In [ ]:
zip_metrics['FICO Accuracy'].mean(),zip_metrics['EnergyScore Accuracy'].mean()

In [ ]:
zip_geojson.head()